# RAG System Exercise - Building a Complete RAG System with Mistral

**IST402 - AI Agents & RAG Systems**

This notebook implements a complete RAG (Retrieval-Augmented Generation) system following the class exercise requirements:

- **Step 1**: Create business-specific system prompts
- **Step 2**: Generate Q&A database using Mistral
- **Step 3**: Implement FAISS vector database for similarity search
- **Step 4**: Create test questions (answerable and unanswerable)
- **Step 5**: Test RAG system performance
- **Step 6**: Evaluate and rank multiple QA models

---

## Setup and Imports

In [ ]:
import sys
import os
import time
import json
import re
import warnings
from typing import Dict, Tuple, List, Any, Optional

warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", message=".*weights.*not initialized.*")
warnings.filterwarnings("ignore", message=".*You should probably TRAIN.*")
os.environ["TRANSFORMERS_VERBOSITY"] = "error"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

print("✅ Environment setup complete!")

In [ ]:
try:
    from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, logging as transformers_logging
    from sentence_transformers import SentenceTransformer
    import torch
    import numpy as np
    import faiss
    transformers_logging.set_verbosity_error()
    print("✅ All required libraries imported successfully!")
except (ImportError, RuntimeError) as e:
    error_msg = str(e)
    if "register_fake" in error_msg or "torch.library" in error_msg:
        print("❌ Dependency version mismatch! Run: pip install --upgrade torch torchvision")
    else:
        print(f"❌ Error: {error_msg}")
        print("Install with: pip install transformers torch sentence-transformers faiss-cpu")
    sys.exit(1)

## Configuration

In [ ]:
MISTRAL_MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"
EMBEDDING_MODEL_ID = "sentence-transformers/all-MiniLM-L6-v2"
SIMILARITY_THRESHOLD = 0.7
BUSINESS_NAME = "TechStart Solutions"
ROLE = "AI Solutions Consultant"

QA_MODELS = [
    "consciousAI/question-answering-generative-t5-v1-base-s-q-c",
    "deepset/roberta-base-squad2",
    "google-bert/bert-large-cased-whole-word-masking-finetuned-squad",
    "gasolsun/DynamicRAG-8B",
    "distilbert-base-uncased-distilled-squad",
    "mrm8488/bert-tiny-finetuned-squadv2",
]

print(f"📋 Business: {BUSINESS_NAME}")
print(f"📋 Role: {ROLE}")
print(f"✅ Configured {len(QA_MODELS)} QA models")

The notebook is organized step-by-step for clarity and learning. Run all cells in order.

## Device Detection & Configuration

Automatically detect if GPU is available and configure accordingly.

## Authentication

Get Hugging Face token from environment variables.

## Load Mistral Model

Load the Mistral model for generating Q&A pairs.

---

# STEP 1: Create System Prompt

Create a system prompt that defines the AI's role and expertise.

---

# STEP 2: Generate Q&A Database

Use Mistral to generate Q&A pairs (7 answerable + 7 unanswerable).

---

# STEP 3: Implement FAISS Vector Database

Convert Q&A database into embeddings and create FAISS index.

---

# STEP 4: Create Test Questions

Generate test questions (answerable and unanswerable).

---

# STEP 5: Test RAG System

Test the RAG system with both question types.

---

# STEP 6: Model Experimentation and Ranking

Test and rank multiple QA models.

In [ ]:
# Execute Step 1
system_prompt = create_system_prompt(BUSINESS_NAME, ROLE)

In [ ]:
# Execute Step 2: Load Mistral and generate Q&A
device_config = get_device_configuration()
hf_token = setup_token()
model, tokenizer = load_mistral_model(MISTRAL_MODEL_ID, hf_token, device_config)
chatbot = create_mistral_pipeline(model, tokenizer, device_config)
answerable_qa, unanswerable_qa = generate_qa_database(chatbot, system_prompt, BUSINESS_NAME)
qa_database = answerable_qa

In [ ]:
# Execute Step 3
embedding_model, faiss_index = implement_faiss_database(qa_database, hf_token)

In [ ]:
# Execute Step 4
print("=" * 60)
print("STEP 4: Preparing Test Questions")
print("=" * 60)
print("\n📚 Using unanswerable pairs from Step 2 as test questions")
print(f"   Unanswerable test questions: {len(unanswerable_qa)}")
print("\n📚 Generating additional answerable test questions...")
additional_answerable = generate_test_questions(chatbot, system_prompt, "answerable", BUSINESS_NAME)
answerable = [qa["question"] for qa in answerable_qa[:5]] + additional_answerable[:2]
unanswerable = [qa["question"] for qa in unanswerable_qa[:7]]
print(f"   Answerable test questions: {len(answerable)}")
print(f"   Unanswerable test questions: {len(unanswerable)}")
print()

In [ ]:
# Execute Step 5
implement_and_test_questions(answerable, unanswerable, embedding_model, faiss_index, qa_database)

In [ ]:
# Execute Step 6
test_questions = [qa["question"] for qa in answerable_qa[:5]]
model_rankings = rank_qa_models(qa_database, embedding_model, faiss_index, hf_token, test_questions=test_questions)

---

## ✅ Exercise Completed!

You have successfully built a complete RAG system with:
- System prompt creation
- Q&A database generation
- FAISS vector database
- Test question generation
- RAG system testing
- QA model evaluation and ranking